In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 2000*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240313194110
0	Validation losses: 0.4851, 150.9633, 151.4483	Best Loss: 151.4483 (0)	Accuracy: 81.24%
1	Validation losses: 0.3858, 114.7893, 115.1751	Best Loss: 115.1751 (0)	Accuracy: 86.44%
2	Validation losses: 0.3853, 98.3381, 98.7234	Best Loss: 98.7234 (0)	Accuracy: 86.08%
3	Validation losses: 0.4005, 88.3363, 88.7368	Best Loss: 88.7368 (0)	Accuracy: 85.52%
4	Validation losses: 0.3570, 81.2738, 81.6308	Best Loss: 81.6308 (0)	Accuracy: 86.44%
5	Validation losses: 0.4428, 76.0323, 76.4751	Best Loss: 76.4751 (0)	Accuracy: 83.88%
6	Validation losses: 0.3158, 71.7337, 72.0495	Best Loss: 72.0495 (0)	Accuracy: 88.26%
7	Validation losses: 0.3490, 68.2710, 68.6199	Best Loss: 68.6199 (0)	Accuracy: 87.02%
8	Validation losses: 0.3268, 65.3753, 65.7021	Best Loss: 65.7021 (0)	Accuracy: 88.16%
9	Validation losses: 0.3227, 62.8496, 63.1724	Best Loss: 63.1724 (0)	Accuracy: 88.10%
10	Validation losses: 0.4019, 60.6372, 61.0392	Best Loss: 61.0392 (0)	Accurac

18	Validation losses: 0.3238, 48.5917, 48.9155	Best Loss: 48.9155 (0)	Accuracy: 87.86%
19	Validation losses: 0.3423, 47.8117, 48.1539	Best Loss: 48.1539 (0)	Accuracy: 87.52%
20	Validation losses: 0.3915, 46.6351, 47.0266	Best Loss: 47.0266 (0)	Accuracy: 85.98%
21	Validation losses: 0.3090, 45.7257, 46.0347	Best Loss: 46.0347 (0)	Accuracy: 89.20%
22	Validation losses: 0.3055, 45.1289, 45.4344	Best Loss: 45.4344 (0)	Accuracy: 89.36%
23	Validation losses: 0.3011, 44.5292, 44.8303	Best Loss: 44.8303 (0)	Accuracy: 88.66%
24	Validation losses: 0.3022, 43.9874, 44.2895	Best Loss: 44.2895 (0)	Accuracy: 89.62%
25	Validation losses: 0.3326, 43.6421, 43.9747	Best Loss: 43.9747 (0)	Accuracy: 87.02%
26	Validation losses: 0.3009, 43.2100, 43.5109	Best Loss: 43.5109 (0)	Accuracy: 89.50%
27	Validation losses: 0.3095, 42.9555, 43.2650	Best Loss: 43.2650 (0)	Accuracy: 89.00%
28	Validation losses: 0.3225, 42.5353, 42.8579	Best Loss: 42.8579 (0)	Accuracy: 90.28%
29	Validation losses: 0.3425, 42.2739, 42.6

113	Validation losses: 0.6370, 36.1089, 36.7460	Best Loss: 36.7460 (0)	Accuracy: 90.56%
114	Validation losses: 0.6362, 36.0948, 36.7310	Best Loss: 36.7310 (0)	Accuracy: 90.68%
115	Validation losses: 0.6499, 36.0824, 36.7323	Best Loss: 36.7310 (1)	Accuracy: 90.60%
116	Validation losses: 0.6509, 36.0709, 36.7217	Best Loss: 36.7217 (0)	Accuracy: 90.58%
117	Validation losses: 0.6561, 36.0605, 36.7166	Best Loss: 36.7166 (0)	Accuracy: 90.84%
118	Validation losses: 0.6597, 36.0526, 36.7123	Best Loss: 36.7123 (0)	Accuracy: 90.64%
119	Validation losses: 0.6577, 36.0466, 36.7043	Best Loss: 36.7043 (0)	Accuracy: 90.76%
120	Validation losses: 0.6658, 36.0411, 36.7069	Best Loss: 36.7043 (1)	Accuracy: 90.68%
121	Validation losses: 0.6666, 36.0350, 36.7016	Best Loss: 36.7016 (0)	Accuracy: 90.62%
122	Validation losses: 0.6690, 36.0286, 36.6976	Best Loss: 36.6976 (0)	Accuracy: 90.58%
123	Validation losses: 0.6737, 36.0227, 36.6963	Best Loss: 36.6963 (0)	Accuracy: 90.54%
124	Validation losses: 0.6764, 3

207	Validation losses: 0.8114, 35.4646, 36.2760	Best Loss: 36.2760 (0)	Accuracy: 90.54%
208	Validation losses: 0.8110, 35.4614, 36.2725	Best Loss: 36.2725 (0)	Accuracy: 90.50%
209	Validation losses: 0.8112, 35.4576, 36.2688	Best Loss: 36.2688 (0)	Accuracy: 90.56%
210	Validation losses: 0.8114, 35.4546, 36.2660	Best Loss: 36.2660 (0)	Accuracy: 90.50%
211	Validation losses: 0.8125, 35.4502, 36.2627	Best Loss: 36.2627 (0)	Accuracy: 90.48%
212	Validation losses: 0.8133, 35.4471, 36.2604	Best Loss: 36.2604 (0)	Accuracy: 90.52%
213	Validation losses: 0.8133, 35.4445, 36.2578	Best Loss: 36.2578 (0)	Accuracy: 90.52%
214	Validation losses: 0.8131, 35.4401, 36.2532	Best Loss: 36.2532 (0)	Accuracy: 90.48%
215	Validation losses: 0.8151, 35.4365, 36.2516	Best Loss: 36.2516 (0)	Accuracy: 90.54%
216	Validation losses: 0.8163, 35.4337, 36.2501	Best Loss: 36.2501 (0)	Accuracy: 90.50%
217	Validation losses: 0.8173, 35.4310, 36.2482	Best Loss: 36.2482 (0)	Accuracy: 90.48%
218	Validation losses: 0.8169, 3

7	Validation losses: 0.3661, 47.0394, 47.4055	Best Loss: 47.4055 (0)	Accuracy: 87.52%
8	Validation losses: 0.3289, 44.9747, 45.3036	Best Loss: 45.3036 (0)	Accuracy: 87.58%
9	Validation losses: 0.3532, 43.1808, 43.5341	Best Loss: 43.5341 (0)	Accuracy: 87.50%
10	Validation losses: 0.3330, 41.6653, 41.9982	Best Loss: 41.9982 (0)	Accuracy: 87.42%
11	Validation losses: 0.3840, 40.3153, 40.6993	Best Loss: 40.6993 (0)	Accuracy: 86.16%
12	Validation losses: 0.3383, 39.0740, 39.4124	Best Loss: 39.4124 (0)	Accuracy: 87.88%
13	Validation losses: 0.3226, 37.9746, 38.2972	Best Loss: 38.2972 (0)	Accuracy: 87.70%
14	Validation losses: 0.3618, 36.9567, 37.3186	Best Loss: 37.3186 (0)	Accuracy: 86.38%
15	Validation losses: 0.3112, 36.1401, 36.4513	Best Loss: 36.4513 (0)	Accuracy: 88.42%
16	Validation losses: 0.3789, 35.1526, 35.5316	Best Loss: 35.5316 (0)	Accuracy: 86.10%
17	Validation losses: 0.2993, 34.2560, 34.5553	Best Loss: 34.5553 (0)	Accuracy: 88.92%
18	Validation losses: 0.3129, 33.4961, 33.8090

...

27	Validation losses: 0.3649, 0.4273, 0.7922	Best Loss: 0.7736 (1)	Accuracy: 89.30%
28	Validation losses: 0.3602, 0.4215, 0.7817	Best Loss: 0.7736 (2)	Accuracy: 90.54%
29	Validation losses: 0.3708, 0.4158, 0.7866	Best Loss: 0.7736 (3)	Accuracy: 90.16%
30	Validation losses: 0.3839, 0.4113, 0.7952	Best Loss: 0.7736 (4)	Accuracy: 90.28%
31	Validation losses: 0.4100, 0.4073, 0.8173	Best Loss: 0.7736 (5)	Accuracy: 90.72%
32	Validation losses: 0.3984, 0.4042, 0.8026	Best Loss: 0.7736 (6)	Accuracy: 90.56%
33	Validation losses: 0.4269, 0.4010, 0.8279	Best Loss: 0.7736 (7)	Accuracy: 90.24%
34	Validation losses: 0.4462, 0.3983, 0.8445	Best Loss: 0.7736 (8)	Accuracy: 90.42%
35	Validation losses: 0.4813, 0.3958, 0.8771	Best Loss: 0.7736 (9)	Accuracy: 90.04%
36	Validation losses: 0.5565, 0.3936, 0.9500	Best Loss: 0.7736 (10)	Accuracy: 89.52%
37	Validation losses: 0.5875, 0.3918, 0.9793	Best Loss: 0.7736 (11)	Accuracy: 90.76%
38	Validation losses: 0.6158, 0.3904, 1.0062	Best Loss: 0.7736 (12)	Accura

36	Validation losses: 0.5092, 0.3936, 0.9028	Best Loss: 0.7604 (10)	Accuracy: 90.52%
37	Validation losses: 0.5766, 0.3917, 0.9683	Best Loss: 0.7604 (11)	Accuracy: 90.56%
38	Validation losses: 0.5993, 0.3903, 0.9896	Best Loss: 0.7604 (12)	Accuracy: 90.70%
39	Validation losses: 0.6448, 0.3893, 1.0342	Best Loss: 0.7604 (13)	Accuracy: 90.64%
40	Validation losses: 0.6538, 0.3889, 1.0427	Best Loss: 0.7604 (14)	Accuracy: 90.78%
41	Validation losses: 0.6696, 0.3882, 1.0578	Best Loss: 0.7604 (15)	Accuracy: 90.90%
42	Validation losses: 0.6924, 0.3873, 1.0797	Best Loss: 0.7604 (16)	Accuracy: 90.82%
43	Validation losses: 0.7002, 0.3863, 1.0865	Best Loss: 0.7604 (17)	Accuracy: 90.84%
44	Validation losses: 0.7062, 0.3854, 1.0915	Best Loss: 0.7604 (18)	Accuracy: 90.56%
45	Validation losses: 0.6455, 0.3848, 1.0303	Best Loss: 0.7604 (19)	Accuracy: 90.48%
46	Validation losses: 0.6583, 0.3846, 1.0429	Best Loss: 0.7604 (20)	Accuracy: 90.40%
Early stopping!
Total running time:	 4002
INFO:tensorflow:Restori

36	Validation losses: 0.5577, 0.2932, 0.8509	Best Loss: 0.6487 (11)	Accuracy: 90.56%
37	Validation losses: 0.6155, 0.2918, 0.9074	Best Loss: 0.6487 (12)	Accuracy: 90.68%
38	Validation losses: 0.6370, 0.2908, 0.9278	Best Loss: 0.6487 (13)	Accuracy: 90.58%
39	Validation losses: 0.6858, 0.2901, 0.9758	Best Loss: 0.6487 (14)	Accuracy: 90.76%
40	Validation losses: 0.7091, 0.2897, 0.9988	Best Loss: 0.6487 (15)	Accuracy: 90.84%
41	Validation losses: 0.7271, 0.2892, 1.0163	Best Loss: 0.6487 (16)	Accuracy: 90.80%
42	Validation losses: 0.7520, 0.2885, 1.0405	Best Loss: 0.6487 (17)	Accuracy: 90.80%
43	Validation losses: 0.7707, 0.2876, 1.0583	Best Loss: 0.6487 (18)	Accuracy: 90.66%
44	Validation losses: 0.7812, 0.2867, 1.0679	Best Loss: 0.6487 (19)	Accuracy: 90.48%
45	Validation losses: 0.7544, 0.2858, 1.0403	Best Loss: 0.6487 (20)	Accuracy: 90.12%
Early stopping!
Total running time:	 3934
INFO:tensorflow:Restoring parameters from ./models/model_20240316133013.ckpt
Batches remaining:     5
Batche

35	Validation losses: 0.5160, 0.2200, 0.7360	Best Loss: 0.5593 (12)	Accuracy: 90.36%
36	Validation losses: 0.5806, 0.2189, 0.7995	Best Loss: 0.5593 (13)	Accuracy: 90.04%
37	Validation losses: 0.6419, 0.2179, 0.8598	Best Loss: 0.5593 (14)	Accuracy: 90.34%
38	Validation losses: 0.6839, 0.2172, 0.9011	Best Loss: 0.5593 (15)	Accuracy: 90.34%
39	Validation losses: 0.7105, 0.2167, 0.9272	Best Loss: 0.5593 (16)	Accuracy: 90.24%
40	Validation losses: 0.7268, 0.2165, 0.9433	Best Loss: 0.5593 (17)	Accuracy: 90.22%
41	Validation losses: 0.7478, 0.2162, 0.9640	Best Loss: 0.5593 (18)	Accuracy: 90.30%
42	Validation losses: 0.7745, 0.2157, 0.9902	Best Loss: 0.5593 (19)	Accuracy: 90.10%
43	Validation losses: 0.8017, 0.2151, 1.0168	Best Loss: 0.5593 (20)	Accuracy: 89.98%
Early stopping!
Total running time:	 3771
INFO:tensorflow:Restoring parameters from ./models/model_20240316154001.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remainin

35	Validation losses: 0.5385, 0.2197, 0.7582	Best Loss: 0.5683 (18)	Accuracy: 90.72%
36	Validation losses: 0.5727, 0.2185, 0.7912	Best Loss: 0.5683 (19)	Accuracy: 90.10%
37	Validation losses: 0.6381, 0.2176, 0.8557	Best Loss: 0.5683 (20)	Accuracy: 90.58%
Early stopping!
Total running time:	 3270
INFO:tensorflow:Restoring parameters from ./models/model_20240316174839.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    11.0400009155%
